<img src="images/csdms_logo.jpg">

# Using a BMI: Coupling Waves and Coastline Evolution Model
This example explores how to use a BMI implementation to couple the Waves component with the Coastline Evolution Model component.

## Links
* [CEM source code](https://github.com/csdms/cem-old): Look at the files that have *deltas* in their name.
* [CEM description on CSDMS](http://csdms.colorado.edu/wiki/Model_help:CEM): Detailed information on the CEM model.

# Interacting with the Coastline Evolution Model BMI using Python

Some magic that allows us to view images within the notebook.

In [ ]:
%pylab inline

Import the `Cem` class, and instantiate it. In Python, a model with a BMI will have no arguments for its constructor. Note that although the class has been instantiated, it's not yet ready to be run. We'll get to that later!

In [ ]:
from cmt.components import Cem, Waves
cem, waves = Cem(), Waves()

Even though we can't run our waves model yet, we can still get some information about it. *Just don't try to run it.* Some things we can do with our model are get the names of the input variables.

In [ ]:
waves.get_output_var_names()

In [ ]:
cem.get_input_var_names()

We can also get information about specific variables. Here we'll look at some info about wave direction. This is the main input of the Cem model. Notice that BMI components always use [CSDMS standard names](http://csdms.colorado.edu/wiki/CSDMS_Standard_Names). The CSDMS Standard Name for wave angle is,

    "sea_surface_water_wave__azimuth_angle_of_opposite_of_phase_velocity"

Quite a mouthful, I know. With that name we can get information about that variable and the grid that it is on (it's actually not a one).

OK. We're finally ready to run the model. Well not quite. First we initialize the model with the BMI **initialize** method. Normally we would pass it a string that represents the name of an input file. For this example we'll pass **None**, which tells Cem to use some defaults.

In [ ]:
cem.initialize(None)
waves.initialize(None)

Here I define a convenience function for plotting the water depth and making it look pretty. You don't need to worry too much about it's internals for this tutorial. It just saves us some typing later on.

In [ ]:
def plot_coast(spacing, z):
    xmin, xmax = 0., z.shape[1] * spacing[1] * 1e-3
    ymin, ymax = 0., z.shape[0] * spacing[0] * 1e-3

    imshow(z, extent=[xmin, xmax, ymin, ymax], origin='lower', cmap='ocean')
    colorbar().ax.set_ylabel('Water Depth (m)')
    xlabel('Along shore (km)')
    ylabel('Cross shore (km)')

It generates plots that look like this. We begin with a flat delta (green) and a linear coastline (y = 3 km). The bathymetry drops off linearly to the top of the domain.

In [ ]:
grid_id = cem.get_var_grid('sea_water__depth')
spacing = cem.get_grid_spacing(grid_id)
shape = cem.get_grid_shape(grid_id)
z = np.empty(shape)
cem.get_value('sea_water__depth', out=z)
plot_coast(spacing, z)

Right now we have waves coming in but no sediment entering the ocean. To add some discharge, we need to figure out where to put it. For now we'll put it on a cell that's next to the ocean.

In [ ]:
water_depth = z[:, 100]
river_mouth = np.where(water_depth > 0.)[0][0]
river_mouth

Allocate memory for the sediment discharge array and set the discharge at the coastal cell to some value.

In [ ]:
qs = np.zeros_like(z)
qs[0, 100] = 750

The CSDMS Standard Name for this variable is:

    "land_surface_water_sediment~bedload__mass_flow_rate"

You can get an idea of the units based on the quantity part of the name. "mass_flow_rate" indicates mass per time. You can double-check this with the BMI method function **get_var_units**.

In [ ]:
cem.get_var_units('land_surface_water_sediment~bedload__mass_flow_rate')

In [ ]:
waves.set_value('sea_shoreline_wave~incoming~deepwater__ashton_et_al_approach_angle_asymmetry_parameter', .3)
waves.set_value('sea_shoreline_wave~incoming~deepwater__ashton_et_al_approach_angle_highness_parameter', .7)
cem.set_value("sea_surface_water_wave__height", 2.)
cem.set_value("sea_surface_water_wave__period", 7.)

Set the bedload flux and run the model.

In [ ]:
for time in xrange(3000):
    waves.update()
    angle = waves.get_value('sea_surface_water_wave__azimuth_angle_of_opposite_of_phase_velocity')

    cem.set_value('sea_surface_water_wave__azimuth_angle_of_opposite_of_phase_velocity', angle)
    cem.set_value('land_surface_water_sediment~bedload__mass_flow_rate', qs)
    cem.update()

cem.get_value('sea_water__depth', out=z)

In [ ]:
plot_coast(spacing, z)

Let's add another sediment source with a different flux and update the model.

In [ ]:
qs[0, 150] = 500
for time in xrange(3750):
    waves.update()
    angle = waves.get_value('sea_surface_water_wave__azimuth_angle_of_opposite_of_phase_velocity')
    cem.set_value('sea_surface_water_wave__azimuth_angle_of_opposite_of_phase_velocity', angle)

    cem.set_value('land_surface_water_sediment~bedload__mass_flow_rate', qs)
    cem.update()
    
cem.get_value('sea_water__depth', out=z)

In [ ]:
plot_coast(spacing, z)

Here we shut off the sediment supply completely.

In [ ]:
qs.fill(0.)
for time in xrange(4000):
    waves.update()
    angle = waves.get_value('sea_surface_water_wave__azimuth_angle_of_opposite_of_phase_velocity')
    cem.set_value('sea_surface_water_wave__azimuth_angle_of_opposite_of_phase_velocity', angle)

    cem.set_value('land_surface_water_sediment~bedload__mass_flow_rate', qs)
    cem.update()
    
cem.get_value('sea_water__depth', out=z)

In [ ]:
plot_coast(spacing, z)